In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
from matplotlib.colors import Normalize
warnings.filterwarnings('ignore')

In [ ]:
pre = "MH130"
drive = 'F'
#species = 'cyno'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
_,base_table_path_MH95,_,_ = h.return_local_paths(drive = drive,pre = 'MH95',add_path = True)

## 06hr vs. 06hr and 24hr vs. 24hr

In [ ]:
df_processed = pd.read_csv(os.path.join(base_table_path_MH95,'MH95_df_DGE_processed_with_prop.csv'),index_col=0)

df_processed_filtered = df_processed[df_processed["comparison_vs_00hr_M"] == df_processed["comparison_vs_00hr_H"]]

df_processed_filtered['nr_FDR_cyno_only'] = df_processed_filtered['nr_FDR_cyno'] - df_processed_filtered['nr_FDR_both']
df_processed_filtered['nr_FDR_human_only'] = df_processed_filtered['nr_FDR_human'] - df_processed_filtered['nr_FDR_both']

df_processed_filtered['log_nr_FDR_both'] = np.log10(df_processed_filtered['nr_FDR_both'])
df_processed_filtered['log_nr_FDR_cyno_only'] = np.log10(df_processed_filtered['nr_FDR_cyno_only'])
df_processed_filtered['log_nr_FDR_human_only'] = np.log10(df_processed_filtered['nr_FDR_human_only'])

df_processed_filtered['significance_0.01'] = np.where(df_processed_filtered['p_value_prop_test'] <= 0.01, '*', ' ')

df_processed_filtered_24hr = df_processed_filtered[df_processed_filtered['comparison_vs_00hr_H']== 'timepoints24hr']

df_processed_filtered_06hr = df_processed_filtered[df_processed_filtered['comparison_vs_00hr_H']== 'timepoints06hr']

df_processed_filtered_06hr.to_csv(os.path.join(base_table_path,pre + '_df_processed_filtered_06hr.csv'))
df_processed_filtered_24hr.to_csv(os.path.join(base_table_path,pre + '_df_processed_filtered_24hr.csv'))

In [ ]:
col_human =  "#e35e28"
col_cyno ="#31c7ba"
col_both = '#6a0dad'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


cell_types_06hr = [f"{cell}_06hr" for cell in df_processed_filtered_06hr["celltype"]]
cell_types_24hr = [f"{cell}_24hr" for cell in df_processed_filtered_24hr["celltype"]]

cell_types_combined = cell_types_06hr + cell_types_24hr


x = np.arange(len(cell_types_combined))
width = 0.28  # Width of bars


fig, ax = plt.subplots(figsize=(14, 6))


bars_06hr_both = ax.bar(x[:len(cell_types_06hr)] + width, df_processed_filtered_06hr["log_nr_FDR_both"], width=width, label="nr_FDR_both_06hr", alpha=0.7, color=col_both)
ax.bar(x[:len(cell_types_06hr)] - width, df_processed_filtered_06hr["log_nr_FDR_cyno_only"], width=width, label="nr_FDR_cyno_06hr_only", alpha=0.5, color=col_cyno)
ax.bar(x[:len(cell_types_06hr)], df_processed_filtered_06hr["log_nr_FDR_human_only"], width=width, label="nr_FDR_human_06hr_only", alpha=0.5, color=col_human)


bars_24hr_both = ax.bar(x[len(cell_types_06hr):] + width, df_processed_filtered_24hr["log_nr_FDR_both"], width=width, label="nr_FDR_both_24hr", alpha=0.7, color=col_both)
ax.bar(x[len(cell_types_06hr):] - width, df_processed_filtered_24hr["log_nr_FDR_cyno_only"], width=width, label="nr_FDR_cyno_24h_onlyr", alpha=0.5, color=col_cyno)
ax.bar(x[len(cell_types_06hr):], df_processed_filtered_24hr["log_nr_FDR_human_only"], width=width, label="nr_FDR_human_24hr_only", alpha=0.5, color=col_human)


ax.axvline(x[len(cell_types_06hr)] - 0.5, color='black', linestyle='-', linewidth=2)




for bar, frac, star in zip(bars_06hr_both, df_processed_filtered_06hr['fraction_same_direction'], df_processed_filtered_06hr['significance_0.01']):
    ax.text(bar.get_x() + bar.get_width() / 2+ 0.01, bar.get_height()-0.4, f"{frac*100:.0f}%", ha='center', va='bottom', rotation=90,fontsize=12)
    ax.text(bar.get_x() + bar.get_width() / 2+ 0.05, bar.get_height() + 0.07, star, ha='center', va='bottom', rotation=90,fontsize=14)

for bar, frac, star in zip(bars_24hr_both, df_processed_filtered_24hr['fraction_same_direction'], df_processed_filtered_24hr['significance_0.01']):
    ax.text(bar.get_x() + bar.get_width() / 2 + 0.01, bar.get_height()-0.4, f"{frac*100:.0f}%", ha='center', va='bottom', rotation=90,fontsize =12)
    ax.text(bar.get_x() + bar.get_width() / 2+ 0.05, bar.get_height() + 0.07, star, ha='center', va='bottom', rotation=90,fontsize =14)

# Labels and title
ax.set_ylabel("log number of genes",fontsize=12)
ax.set_xticks(x)
ax.tick_params(axis='y', labelsize=12)
ax.set_xticklabels(cell_types_combined, rotation=90)
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
#ax.grid(axis='y', linestyle='--', alpha=0.4)
plt.ylim([0, 4])
plt.savefig(os.path.join(base_plots_path,pre + '_DGE_coenrichment_plot_main.pdf'),bbox_inches='tight')

plt.show()

## 06hr vs. 24hr and 24hr vs. 06hr

In [ ]:
df_processed = pd.read_csv(os.path.join(base_table_path_MH95,'MH95_df_DGE_processed_with_prop.csv'),index_col=0)

df_processed_filtered = df_processed[df_processed["comparison_vs_00hr_M"] != df_processed["comparison_vs_00hr_H"]]


df_processed_filtered['nr_FDR_cyno_only'] = df_processed_filtered['nr_FDR_cyno'] - df_processed_filtered['nr_FDR_both']
df_processed_filtered['nr_FDR_human_only'] = df_processed_filtered['nr_FDR_human'] - df_processed_filtered['nr_FDR_both']

df_processed_filtered['log_nr_FDR_both'] = np.log10(df_processed_filtered['nr_FDR_both'])
df_processed_filtered['log_nr_FDR_cyno_only'] = np.log10(df_processed_filtered['nr_FDR_cyno_only'])
df_processed_filtered['log_nr_FDR_human_only'] = np.log10(df_processed_filtered['nr_FDR_human_only'])

df_processed_filtered['significance_0.01'] = np.where(df_processed_filtered['p_value_prop_test'] <= 0.01, '*', ' ')


df_processed_filtered_H_24hr_M_06hr =  df_processed_filtered[df_processed_filtered['comparison_vs_00hr_H']== 'timepoints24hr']

df_processed_filtered_M_24hr_H_06hr =  df_processed_filtered[df_processed_filtered['comparison_vs_00hr_M']== 'timepoints24hr']

df_processed_filtered_H_24hr_M_06hr.to_csv(os.path.join(base_table_path,pre + '_df_processed_filtered_H_24hr_M_06hr.csv'))
df_processed_filtered_M_24hr_H_06hr.to_csv(os.path.join(base_table_path,pre + '_df_processed_filtered_M_24hr_H_06hr.csv'))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define cell types and combine them into one list for the x-axis
cell_types_H_24hr_M_06hr = [f"{cell}_H_24hr_M_06hr" for cell in df_processed_filtered_H_24hr_M_06hr["celltype"]]
cell_types_M_24hr_H_06hr = [f"{cell}_M_24hr_H_06hr" for cell in df_processed_filtered_M_24hr_H_06hr["celltype"]]
cell_types_combined = cell_types_H_24hr_M_06hr + cell_types_M_24hr_H_06hr

# Generate new x positions for H_24hr_M_06hr and M_24hr_H_06hr data
x = np.arange(len(cell_types_combined))
width = 0.28  # Width of bars

# Create figure and axis
fig, ax = plt.subplots(figsize=(14, 6))

# Plot bars for H_24hr_M_06hr
bars_H_24hr_M_06hr_both = ax.bar(x[:len(cell_types_H_24hr_M_06hr)] + width, df_processed_filtered_H_24hr_M_06hr["log_nr_FDR_both"], width=width, label="nr_FDR_both_H_24hr_M_06hr", alpha=0.7, color=col_both)
ax.bar(x[:len(cell_types_H_24hr_M_06hr)] - width, df_processed_filtered_H_24hr_M_06hr["log_nr_FDR_cyno_only"], width=width, label="nr_FDR_cyno_H_24hr_M_06hr_only", alpha=0.5, color=col_cyno)
ax.bar(x[:len(cell_types_H_24hr_M_06hr)], df_processed_filtered_H_24hr_M_06hr["log_nr_FDR_human_only"], width=width, label="nr_FDR_human_H_24hr_M_06hr_only", alpha=0.5, color=col_human)

# Plot bars for M_24hr_H_06hr
bars_M_24hr_H_06hr_both = ax.bar(x[len(cell_types_H_24hr_M_06hr):] + width, df_processed_filtered_M_24hr_H_06hr["log_nr_FDR_both"], width=width, label="nr_FDR_both_M_24hr_H_06hr", alpha=0.7, color=col_both)
ax.bar(x[len(cell_types_H_24hr_M_06hr):] - width, df_processed_filtered_M_24hr_H_06hr["log_nr_FDR_cyno_only"], width=width, label="nr_FDR_cyno_M_24hr_H_06hr", alpha=0.5, color=col_cyno)
ax.bar(x[len(cell_types_H_24hr_M_06hr):], df_processed_filtered_M_24hr_H_06hr["log_nr_FDR_human_only"], width=width, label="nr_FDR_human_M_24hr_H_06hr", alpha=0.5, color=col_human)

# Draw a vertical line to separate H_24hr_M_06hr from M_24hr_H_06hr data
ax.axvline(x[len(cell_types_H_24hr_M_06hr)] - 0.5, color='black', linestyle='-', linewidth=2)

# Annotate bars with 'fraction_same_direction' values and 'significance_star'
for bar, frac, star in zip(bars_H_24hr_M_06hr_both, df_processed_filtered_H_24hr_M_06hr['fraction_same_direction'], df_processed_filtered_H_24hr_M_06hr['significance_0.01']):
    ax.text(bar.get_x() + bar.get_width() / 2 + 0.01, bar.get_height() -0.4, f"{frac*100:.0f}%", ha='center', va='bottom', rotation=90, fontsize=12)
    ax.text(bar.get_x() + bar.get_width() / 2 + 0.05, bar.get_height() + 0.07, star, ha='center', va='bottom', rotation=90, fontsize=14)

for bar, frac, star in zip(bars_M_24hr_H_06hr_both, df_processed_filtered_M_24hr_H_06hr['fraction_same_direction'], df_processed_filtered_M_24hr_H_06hr['significance_0.01']):
    ax.text(bar.get_x() + bar.get_width() / 2 + 0.01, bar.get_height() -0.4, f"{frac*100:.0f}%", ha='center', va='bottom', rotation=90, fontsize=12)
    ax.text(bar.get_x() + bar.get_width() / 2 + 0.05, bar.get_height() + 0.07, star, ha='center', va='bottom', rotation=90, fontsize=14)

# Labels and title
ax.set_ylabel("log number of genes", fontsize=12)
ax.set_xticks(x)
ax.tick_params(axis='y', labelsize=12)
ax.set_xticklabels(cell_types_combined, rotation=90)
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
# ax.grid(axis='y', linestyle='--', alpha=0.4)
plt.ylim([0, 4])

# Assuming 'base_plots_path' and 'pre' are defined
plt.savefig(os.path.join(base_plots_path, pre + '_DGE_coenrichment_plot_supp.pdf'), bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()